### Run filterPairTgapsMetric for proposalId=1 WFD region opsim v1.5

consider gaps between any filter

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sn

import healpy as hp
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.metricBundles as metricBundles
import lsst.sims.maf.db as db



In [57]:
plt.rc('legend', fontsize=20) # using a size in points
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
plt.rcParams['axes.labelsize'] = 16

In [143]:
# load opsim database
dbpath = "/home/idies/workspace/lsst_cadence/FBS_1.5/"  # path to all opsim databases

dbpath_v17 = "/home/idies/workspace/lsst_cadence/FBS_1.7/"


# output directory
dataRawDir = '/home/idies/workspace/Temporary/lixl/scratch/outDir/dataRaw/'

outDir = '/home/idies/workspace/Temporary/lixl/scratch/outDir/resultsDb/'
resultsDb = db.ResultsDb(outDir=outDir)



In [144]:
# get the name of all opsim dbs 
import glob

workpath = os.getcwd()
#workpath = '/home/idies/workspace/Storage/lixl/persistent/LSST_OpSim/unknowns'

os.chdir(dbpath)  # change to opsim database directory
dblist_all = glob.glob('*.db') 

os.chdir(dbpath_v17)  # change to opsim database directory
dblist_all_v17 = glob.glob('*.db') 

os.chdir(workpath) # change back to work directory
dblist_all.sort()
dblist_all_v17.sort()


In [149]:
dblist_all += dblist_all_v17

In [151]:
len( dblist_all )

160

In [60]:
def RADec2pix(nside, ra, dec, degree=True):
    """
    Calculate the nearest healpixel ID of an RA/Dec array, assuming nside.

    Parameters
    ----------
    nside : int
        The nside value of the healpix grid.
    ra : numpy.ndarray
        The RA values to be converted to healpix ids, in degree by default.
    dec : numpy.ndarray
        The Dec values to be converted to healpix ids, in degree by default.

    Returns
    -------
    numpy.ndarray
        The healpix ids.
    """
    if degree:
        ra = np.radians(ra) # change to radians
        dec = np.radians(dec)
    
    lat = np.pi/2. - dec
    hpid = hp.ang2pix(nside, lat, ra )
    return hpid



In [109]:
from lsst.sims.maf.utils import m52snr

class filterPairTGapsMetric(metrics.BaseMetric):
    """
    returns
    N_v * np.exp(-Dkl) for each fields
    
    Parameters:
        colname: 
        fltpair: filter pair, eg ['r', 'i']
        snr_lim: list, signal to noise ratio (fiveSigmaDepth) threshold for fltpair, default [5, 5]
        filename: output a csv table for time gaps of each field
    
    """

    def __init__(self, colname=['observationStartMJD', 'filter', 'fiveSigmaDepth'], fltpair=['r', 'i'], tmin=0, tmax=1.5/24, bins=50, mag_lim=[18, 18],
                 save_dT=False, allgaps=True, filename=None, dataout=True, **kwargs):
        self.colname = colname
        self.filename = filename
        self.fltpair = fltpair
        self.mag_lim = mag_lim
        self.dataout = dataout
        self.tmin = tmin
        self.tmax = tmax
        self.bins = bins
        self.save_dT = save_dT
        self.allgaps = allgaps
        self.Nrun = 0   # record the how many time run run()
        if filename!=None:
            if os.path.isfile(filename):
                # rm old file
                os.system("rm {}".format(filename))
                
        if self.dataout:
            super().__init__(col=self.colname, metricDtype='object', **kwargs)
        else:
            super().__init__(col=self.colname, metricDtype='float', **kwargs)
    
    def save_to_file(self, dic, filename="test_pkl.pkl"):
        '''save dict item to pickle file'''
        
        #df = self.load_from_pkl(filename)

        #df = df.append(pd.DataFrame(dic), ignore_index=True)

        #df.to_pickle(filename)
        df = pd.DataFrame(dic)
        with open(filename, 'a') as f:
            df.to_csv(f, header=f.tell()==0, index=None)
      
    def run(self, dataSlice, slicePoint=None):
        
        # return all possible time gaps for each fields
        
        f0 = self.fltpair[0]
        f1 = self.fltpair[1]
        
        #check input config
        #print(f0, f1, self.tmin, self.tmax, self.mag_lim)
            
        # sort dataSlice
        
        idx0 = ( dataSlice['filter'] == f0 ) & ( dataSlice['fiveSigmaDepth'] > self.mag_lim[0])
        idx1 = ( dataSlice['filter'] == f1 ) & ( dataSlice['fiveSigmaDepth'] > self.mag_lim[1])
        
        timeCol0 = dataSlice['observationStartMJD'][idx0]
        timeCol1 = dataSlice['observationStartMJD'][idx1]

        timeCol0 = timeCol0.reshape((len(timeCol0), 1))
        timeCol1 = timeCol1.reshape((len(timeCol1), 1))
        
        
        # calculate time gaps matrix
        diffmat = np.subtract(timeCol0, timeCol1.T)
        
        if self.allgaps:
            # collect all time gaps
            
            diffmat = np.abs( diffmat )
            if f0==f1:
                # get only triangle part
                dt_tri = np.tril(diffmat, -1)
                dT = dt_tri[dt_tri!=0]    # flatten lower triangle 
            else:
                dT = diffmat.flatten()
                
        else:
            # collect only nearest 

            if f0==f1:
                # get diagonal ones nearest nonzero, offset=1
                
                dT = np.diagonal(diffmat, offset=1)
            else:
                # keep only negative ones

                masked_ar = np.ma.masked_where(diffmat>=0, diffmat, )
                left_ar = np.max(masked_ar, axis=1)
                dT_left = -left_ar.data[~left_ar.mask]
                
                # keep only positive ones
                masked_ar = np.ma.masked_where(diffmat<=0, diffmat)
                right_ar = np.min(masked_ar, axis=1)
                dT_right = right_ar.data[~right_ar.mask]
                
                dT = np.concatenate([dT_left.flatten(), dT_right.flatten()])

            
        dT_tlim = dT[(dT>tmin)&(dT<tmax)] 
        Nv = len(dT_tlim)
        
        # print(self.Nrun, np.min(dataSlice['fiveSigmaDepth'][idx0]), np.min(dataSlice['fiveSigmaDepth'][idx1]),)
        self.Nrun += 1
        # write results to csv file
        fieldRA = np.mean(dataSlice['fieldRA']) ,
        fieldDec = np.mean(dataSlice['fieldDec']),
        
        if self.save_dT:
            dic = {'Nrun': self.Nrun, 
                'pixId': RADec2pix(nside=16, ra=fieldRA, dec=fieldDec)[0],
                'Nv': Nv,
                'dT_lim': dT_tlim
                  }
        else:
            dic = {'Nrun': self.Nrun, 
                'pixId': RADec2pix(nside=16, ra=fieldRA, dec=fieldDec)[0],
                'Nv': Nv,
                # 'dT_lim': dT_tlim
                  }
            
        if self.filename!=None:
            self.save_to_file(dic, filename=self.filename)
        
        if self.dataout:
            # return dT
            result = dic
            return result
        else:
        #    f0 = self.fltpair[0]
        #    f1 = self.fltpair[1]
            result = np.min(dT) if len(dT)!=0 else np.inf
            return float(result) 



In [139]:
from lsst.sims.maf.utils import m52snr

class anyfilterPairTGapsMetric(metrics.BaseMetric):
    """
    returns
    N_v * np.exp(-Dkl) for each fields
    
    Parameters:
        colname: 
        fltpair: filter pair, eg ['r', 'i']
        snr_lim: list, signal to noise ratio (fiveSigmaDepth) threshold for fltpair, default [5, 5]
        filename: output a csv table for time gaps of each field
    
    """

    def __init__(self, colname=['observationStartMJD', 'filter', 'fiveSigmaDepth'], tmin=0, tmax=1.5/24, 
                 save_dT=False, allgaps=True, filename=None, dataout=True, **kwargs):
        self.colname = colname
        self.filename = filename
        self.dataout = dataout
        self.tmin = tmin
        self.tmax = tmax
        self.save_dT = save_dT
        self.allgaps = allgaps
        self.Nrun = 0   # record the how many time run run()
        if filename!=None:
            if os.path.isfile(filename):
                # rm old file
                os.system("rm {}".format(filename))
                
        if self.dataout:
            super().__init__(col=self.colname, metricDtype='object', **kwargs)
        else:
            super().__init__(col=self.colname, metricDtype='float', **kwargs)
    
    def save_to_file(self, dic, filename="test_pkl.pkl"):
        '''save dict item to pickle file'''
        
        #df = self.load_from_pkl(filename)

        #df = df.append(pd.DataFrame(dic), ignore_index=True)

        #df.to_pickle(filename)
        df = pd.DataFrame(dic)
        with open(filename, 'a') as f:
            df.to_csv(f, header=f.tell()==0, index=None)
      
    def run(self, dataSlice, slicePoint=None):
        
        # return all possible time gaps for each fields

        #check input config
        #print(f0, f1, self.tmin, self.tmax, self.mag_lim)
            
        # sort dataSlice
        #np.sort(dataSlice['observationStartMJD'])
        
        #idx0 = ( dataSlice['filter'] == f0 ) & ( dataSlice['fiveSigmaDepth'] > self.mag_lim[0])
        #idx1 = ( dataSlice['filter'] == f1 ) & ( dataSlice['fiveSigmaDepth'] > self.mag_lim[1])
        
        timeCol0 = np.sort( dataSlice['observationStartMJD'] )
        timeCol1 = np.sort( dataSlice['observationStartMJD'] )

        timeCol0 = timeCol0.reshape((len(timeCol0), 1))
        timeCol1 = timeCol1.reshape((len(timeCol1), 1))
        
        
        # calculate time gaps matrix
        diffmat = np.subtract(timeCol0, timeCol1.T)
        

        if self.allgaps:
            # collect all time gaps
            
            diffmat = np.abs( diffmat )
            # get only triangle part
            dt_tri = np.tril(diffmat, -1)
            dT = dt_tri[dt_tri!=0]    # flatten lower triangle 

        else:
            # collect only nearest 

            # get diagonal ones nearest nonzero, offset=1
            
            #print("counting nearest gaps")
            diffmat = np.abs( diffmat )

            dT = np.diagonal(diffmat, offset=1)
        
        dT_tlim = dT[(dT>tmin)&(dT<tmax)] 

        #print(dT_tlim)
        Nv = len(dT_tlim)
        
        # print(self.Nrun, np.min(dataSlice['fiveSigmaDepth'][idx0]), np.min(dataSlice['fiveSigmaDepth'][idx1]),)
        self.Nrun += 1
        # write results to csv file
        fieldRA = np.mean(dataSlice['fieldRA']) ,
        fieldDec = np.mean(dataSlice['fieldDec']),
        
        if self.save_dT:
            dic = {'Nrun': self.Nrun, 
                'pixId': RADec2pix(nside=16, ra=fieldRA, dec=fieldDec)[0],
                'Nv': Nv,
                'dT_lim': dT_tlim
                  }
        else:
            dic = {'Nrun': self.Nrun, 
                'pixId': RADec2pix(nside=16, ra=fieldRA, dec=fieldDec)[0],
                'Nv': Nv,
                # 'dT_lim': dT_tlim
                  }
            
        if self.filename!=None:
            self.save_to_file(dic, filename=self.filename)
        
        if self.dataout:
            # return dT
            result = dic
            return result
        else:
        #    f0 = self.fltpair[0]
        #    f1 = self.fltpair[1]
            result = np.min(dT) if len(dT)!=0 else np.inf
            return float(result) 




In [137]:
def runMetric(dbname, fltpair, ipix_ms=None, nside=16, sqlstr=None,
              colname=['observationStartMJD', 'filter', 'fiveSigmaDepth'],
              tmin=0, tmax=1.5/24, bins=50, mag_lim=[18, 18],
              filename=None, dataout=True, **kwargs
             ):
    """run filterPairTGapsMetric
    dbname: name of opsim database
    fltpair: filter pair, eg: 'ri'
    ipix_ms: pixIds of footprint
    """
    opsdb = db.OpsimDatabase(dbpath+dbname)
    
    #fltpair = ['r', 'i']
    
    if ipix_ms!=None:
        # use UserPointsSlicer for minisurvey
        ra, dec = pix2radec(nside=nside, ipix=ipix_ms)
        slicer = slicers.UserPointsSlicer(ra, dec, latLonDeg=True, badval=0)
    else:
        # use HealpixSlicer 
        slicer = slicers.HealpixSlicer(nside=nside)
    
    # metric, slicer, constraint
    #metric = filterPairTGapsMetric( colname=colname, fltpair=fltpair, tmin=tmin, tmax=tmax, bins=bins, mag_lim=mag_lim,
    #             filename=filename, dataout=dataout, **kwargs)
    
    metric = anyfilterPairTGapsMetric( colname=colname, tmin=tmin, tmax=tmax, 
                 filename=filename, dataout=dataout,  **kwargs)
    
    
    #sqlconstraint = 'filter in ("{}", "{}")'.format(*fltpair)
    sqlconstraint = sqlstr

    #if sqlstr!=None:
    #   sqlconstraint += ' and {}'.format(sqlstr)
        
    # bundle
    newMetricSky = metricBundles.MetricBundle(metric, slicer, sqlconstraint)
    
    # group bundle
    bundleDict = {'newMetricSky':newMetricSky}
    group = metricBundles.MetricBundleGroup(bundleDict, opsdb, outDir=outDir, resultsDb=resultsDb)
    
    # run
    group.runAll()
    #group.plotAll(closefigs = False)
    return newMetricSky



In [63]:
fltpairs = ['uu', 'ug', 'ur', 'ui', 'uz', 'uy', 'gg', 'gr', 'gi', 'gz', 'gy',
           'rr', 'ri', 'rz', 'ry', 'ii', 'iz', 'iy', 'zz', 'zy', 'yy' ]

In [152]:
%%time 
# run all filter pairs for all opsims 

nside = 16
# select mini survey footprint 
msname = 'WFD' # minisurvey label used to save files

if msname == 'LMC':
    ipix_ms = ipix_LMC
elif msname=='SMC':
    ipix_ms = ipix_SMC
else:
    ipix_ms = None

#sqlstr = "night<4000 and proposalId=1"
#sqlstr = None
sqlstr = "proposalId=1"
#sqlstr = "note not like '%DD%'"


magdic = {'u':18, 'g':18, 'r':18, 'i':18, 'z':18, 'y':18}

#--------------------------------
# empty metricArr 
#diclist = []
#for Nrun in range( len(ipix_ms) ):
#    dic = {'Nrun': Nrun, 
#                'pixId': ipix_ms[Nrun],
#                'Nv': 0,
#                'Dkl': 0,
#                'FoM_i': 0,
#                'dT_lim': []
#                  }
#    diclist.append(dic)
#arr_empty = np.array(diclist)
#df_empty = pd.DataFrame.from_records( arr_empty )
#--------------------------------

for dbname in dblist_all:
    dataRaw = {}
    dataRaw[dbname] = {}
    for fltpair in ['any',]: 
        
        print(">>> ", dbname, fltpair)

        dataRaw[dbname][fltpair] = {}
        
        tmin =  5/60/60/24 # in days
        tmax = 3e8/60./60./24    
            
        metricSky = runMetric(dbname, fltpair=fltpair, sqlstr=sqlstr, nside=nside, ipix_ms=ipix_ms, 
                                 tmin=tmin, tmax=tmax, bins=None,
                                 filename=None, save_dT=True, dataout=True, allgaps=False);
            
        arr = metricSky.metricValues.data 
                
        df = pd.DataFrame.from_records( arr[ arr != None ] )
                    
        dataRaw[dbname][fltpair] = df[['pixId', 'Nv', 'dT_lim']] 
        
        # save raw data to pickle
        pd.DataFrame( dataRaw ).to_pickle(dataRawDir+'/tgaps_any_{}_{}.pkl'.format(dbname,msname))




>>>  agnddf_v1.5_10yrs.db any
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint proposalId=1 for columns ['filter', 'fieldDec', 'observationStartMJD', 'fiveSigmaDepth', 'fieldRA']
Found 1838664 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  alt_dust_v1.5_10yrs.db any
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint proposalId=1 for columns ['filter', 'fieldDec', 'observationStartMJD', 'fiveSigmaDepth', 'fieldRA']
Found 1719370 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  alt_roll_mod2_dust_sdf_0.20_v1.5_10yrs.db any
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint proposalId=1 for columns ['fi

Found 2016721 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  filterdist_indx2_v1.5_10yrs.db any
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint proposalId=1 for columns ['filter', 'fieldDec', 'observationStartMJD', 'fiveSigmaDepth', 'fieldRA']
Found 2019643 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  filterdist_indx3_v1.5_10yrs.db any
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint proposalId=1 for columns ['filter', 'fieldDec', 'observationStartMJD', 'fiveSigmaDepth', 'fieldRA']
Found 2019489 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  filterdist_indx4_v1.5_10yrs.db any
Healpix 

Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  goodseeing_gri_v1.5_10yrs.db any
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint proposalId=1 for columns ['filter', 'fieldDec', 'observationStartMJD', 'fiveSigmaDepth', 'fieldRA']
Found 1850251 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  goodseeing_griz_v1.5_10yrs.db any
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint proposalId=1 for columns ['filter', 'fieldDec', 'observationStartMJD', 'fiveSigmaDepth', 'fieldRA']
Found 1846740 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  goodseeing_gz_v1.5_10yrs.db any
Healpix slicer using NSIDE=16, approximate resolution 219.8711

>>>  third_obs_pt45v1.5_10yrs.db any
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint proposalId=1 for columns ['filter', 'fieldDec', 'observationStartMJD', 'fiveSigmaDepth', 'fieldRA']
Found 1854089 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  third_obs_pt60v1.5_10yrs.db any
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint proposalId=1 for columns ['filter', 'fieldDec', 'observationStartMJD', 'fiveSigmaDepth', 'fieldRA']
Found 1855567 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  third_obs_pt90v1.5_10yrs.db any
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint proposalId=1 for columns ['filt

Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint proposalId=1 for columns ['filter', 'fieldDec', 'observationStartMJD', 'fiveSigmaDepth', 'fieldRA']
Found 1854673 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  wfd_depth_scale0.95_noddf_v1.5_10yrs.db any
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint proposalId=1 for columns ['filter', 'fieldDec', 'observationStartMJD', 'fiveSigmaDepth', 'fieldRA']
Found 2026223 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  wfd_depth_scale0.95_v1.5_10yrs.db any
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint proposalId=1 for columns ['filter', 'fieldDec', 'o

OSError: Sqlite database file "/home/idies/workspace/lsst_cadence/FBS_1.5/baseline_nexp1_v1.7_10yrs.db" not found.

In [154]:
%%time

sqlstr = "night<800 and proposalId=1"
# for v17
dbpath = dbpath_v17
for dbname in dblist_all_v17:
    dataRaw = {}
    dataRaw[dbname] = {}
    for fltpair in ['any',]: 
        
        print(">>> ", dbname, fltpair)

        dataRaw[dbname][fltpair] = {}
        
        tmin =  5/60/60/24 # in days
        tmax = 3e8/60./60./24    
            
        metricSky = runMetric(dbname, fltpair=fltpair, sqlstr=sqlstr, nside=nside, ipix_ms=ipix_ms, 
                                 tmin=tmin, tmax=tmax, bins=None,
                                 filename=None, save_dT=True, dataout=True, allgaps=True);
            
        arr = metricSky.metricValues.data 
                
        df = pd.DataFrame.from_records( arr[ arr != None ] )
                    
        dataRaw[dbname][fltpair] = df[['pixId', 'Nv', 'dT_lim']] 
        
        # save raw data to pickle
        pd.DataFrame( dataRaw ).to_pickle(dataRawDir+'/tgaps_any_all_{}_{}.pkl'.format(dbname,msname))




>>>  baseline_nexp1_v1.7_10yrs.db any
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint night<800 and proposalId=1 for columns ['filter', 'fieldDec', 'observationStartMJD', 'fiveSigmaDepth', 'fieldRA']
Found 391714 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  baseline_nexp2_v1.7_10yrs.db any
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint night<800 and proposalId=1 for columns ['filter', 'fieldDec', 'observationStartMJD', 'fiveSigmaDepth', 'fieldRA']
Found 361712 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  cadence_drive_gl100_gcbv1.7_10yrs.db any
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with const

Found 361180 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  euclid_dither5_v1.7_10yrs.db any
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint night<800 and proposalId=1 for columns ['filter', 'fieldDec', 'observationStartMJD', 'fiveSigmaDepth', 'fieldRA']
Found 361778 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  footprint_0_v1.710yrs.db any
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint night<800 and proposalId=1 for columns ['filter', 'fieldDec', 'observationStartMJD', 'fiveSigmaDepth', 'fieldRA']
Found 323922 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  footprint_1_v1.710yrs.db a

Found 359204 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  rolling_nm_scale0.6_nslice3_v1.7_10yrs.db any
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint night<800 and proposalId=1 for columns ['filter', 'fieldDec', 'observationStartMJD', 'fiveSigmaDepth', 'fieldRA']
Found 359410 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  rolling_nm_scale0.8_nslice2_v1.7_10yrs.db any
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint night<800 and proposalId=1 for columns ['filter', 'fieldDec', 'observationStartMJD', 'fiveSigmaDepth', 'fieldRA']
Found 359135 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>

KeyboardInterrupt: 